# Install and import dependancies

In [ ]:
!pip install -U -q transformers datasets accelerate peft trl bitsandbytes wandb

In [ ]:
from google.colab import userdata

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('huggingface')
hf_token = userdata.get('HF_TOKEN')

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

#Download model

In [ ]:
base_model = "betterMateusz/Llama-2-7b-chat-finetune"

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    force_download=True,
    # cache_dir=None,
)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#Download datasets

In [ ]:
dataset = load_dataset("betterMateusz/paragraphs")
dataset2 = load_dataset("betterMateusz/SAT_Writting_Reading_Assessment_Question_Bank")
# dataset2

In [ ]:
for text in dataset['train']['input']:
  print("*"*20)
  print(text)

# Create a system prompt

In [ ]:
system_prompt = "Please generate 1 grammar question with answers based on the passage."

def format_for_finetuning(row):
    return {
        "input": f"""
<s>[INST]<<SYS>>
{ system_prompt }
<</SYS>>

Passage: {row['passage']}[/INST]
Question: {row['question']}
Choices: A) {row['choice_A']} B) {row['choice_B']} C) {row['choice_C']} D) {row['choice_D']}
Correct Answer: {row['correct_answer']} </s>""",

    }
# dataset = load_dataset("betterMateusz/SAT_Writting_Reading_Assessment_Question_Bank")
train = [format_for_finetuning(row) for row in dataset2['train']]
train_SAT = Dataset.from_list(train)
print(train_SAT[0]['input'])



In [ ]:
# system_prompt = "Please generate a completely different grammar question with answers based on the passage."
generated_problems = []
for id, sentence in enumerate(dataset['train']['input']):
  prompt = train_SAT[id]['input'] + f"""
<s>[INST]<<SYS>>
{ system_prompt }
<</SYS>>

Passage: {sentence}[/INST]
"""
  # print(prompt)
  # break
  output = model.generate(tokenizer(prompt, return_tensors="pt").input_ids, max_length=512, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  generated_problems.append(generated_text)
  print(generated_text)
  print("*"*20)
  if len(generated_problems) == 20:
    break


<s>[INST]<<SYS>>
Please generate 1 grammar question with answers based on the passage.
<</SYS>>

Passage: Studying how workload affects productivity, Maryam Kouchaki and colleagues found that people who chose to do relatively
easy tasks ﬁrst were less ______ compared to those who did hard tasks ﬁrst. Finishing easy tasks gave participants a sense
of accomplishment, but those who tackled hard tasks ﬁrst actually became more skilled and productive workers over time.[/INST]
Question: Which choice completes the text with the most logical and precise word or phrase?
Choices: A) secretive B) eﬃcient C) outgoing D) unsympathetic
Correct Answer: B
<s>[INST]<<SYS>>
Please generate 1 grammar question with answers based on the passage.
<</SYS>>

Passage: Whatever you do, let nothing tempt you to relate more than once any grievance you may have. Nothing of course is more poisonous to the aggrieved one than to stifle his grievance absolutely. Once, and once only, he may produce it to his friends. 

ValueError: Input length of input_ids is 516, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
import re

def is_text_normal(text):
    # Check if the text contains only normal letters, numbers, punctuation, and spaces
    if not re.match(r'^[a-zA-Z0-9\s.,!?\'"-]+$', text):
        return False

    # Check if the text contains multiple spaces between words
    if re.search(r'\s{2,}', text):
        return False

    # Check if the text is not only uppercase
    if text.isupper():
        return False

    return True

randomsentences = []
for text in randomsentences
  if len(text) > 350 and is_text_normal(text) and len(text) < 600:
    print("*"*20)
    print(text)
    new_dataset.append(text)



********************
Whatever you do, let nothing tempt you to relate more than once any grievance you may have. Nothing of course is more poisonous to the aggrieved one than to stifle his grievance absolutely. Once, and once only, he may produce it to his friends. I shall be blamed, perhaps, for making even this slight concession. Please be careful, therefore, not to abuse it
********************
with the power of action, how natural the exclamation, O "Lord, how manifold are thy works! in wisdom hast thou made them all." Under these considerations, that they are the work of the same great, good, and Almighty hand that formed us, and that they are all capable of feeling pleasure and pain, surely every little child, as well as older person, ought carefully to avoid every kind of cruelty to any kind of creature, great or small.
********************
I am prepared to meet rather with rejection than with approval. Neither the historian nor the psychologist will be pleased. Moreover, I am w

In [ ]:
train_SAT = Dataset.from_dict({"input": new_dataset})

train_SAT.push_to_hub("betterMateusz/paragraphs")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/268 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/betterMateusz/paragraphs/commit/98dca0153fd5036dc4f89074b4261e0d691991c9', commit_message='Upload dataset', commit_description='', oid='98dca0153fd5036dc4f89074b4261e0d691991c9', pr_url=None, pr_revision=None, pr_num=None)